<a href="https://colab.research.google.com/github/Hiromi06/machine-translation/blob/main/BERT_sample_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
def load_data(file_path):
  with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
  return [line.strip() for line in lines]

sample_en_train_path = '/content/drive/MyDrive/machine_learning/sample_en_train.txt'
sample_en_test_path = '/content/drive/MyDrive/machine_learning/sample_en_test.txt'
sample_ja_train_path = '/content/drive/MyDrive/machine_learning/sample_ja_train.txt'
sample_ja_test_path = '/content/drive/MyDrive/machine_learning/sample_ja_test.txt'

sample_en_train = load_data(sample_en_train_path)
sample_en_test = load_data(sample_en_test_path)
sample_ja_train = load_data(sample_ja_train_path)
sample_ja_test = load_data(sample_ja_test_path)

#print(sample_en_train[:6])

In [5]:
from transformers import BertTokenizer, BertConfig, BertForMaskedLM, AdamW, get_linear_schedule_with_warmup

model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
config = BertConfig.from_pretrained(model_name)
bert_masked_model = BertForMaskedLM.from_pretrained(model_name, config=config).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def batch_encode(text_list, tokenizer, batch_size=32):
    encoded_batches = []
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i + batch_size]
        encoded_batch = tokenizer(batch, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
        encoded_batches.append(encoded_batch.to(device))
    return encoded_batches

In [7]:
en_train_encoding = batch_encode(sample_en_train, tokenizer)
en_test_encoding = batch_encode(sample_en_test, tokenizer)
ja_train_encoding = batch_encode(sample_ja_train, tokenizer)
ja_test_encoding = batch_encode(sample_ja_test, tokenizer)

In [8]:
# concatenate encoded batches

def concatenate_batches(encoded_batches):
    input_ids = torch.cat([batch['input_ids'] for batch in encoded_batches])
    attention_masks = torch.cat([batch['attention_mask'] for batch in encoded_batches])
    return input_ids, attention_masks

train_input_ids, train_attention_masks = concatenate_batches(en_train_encoding)
val_input_ids, val_attention_masks = concatenate_batches(en_test_encoding)
train_labels, _ = concatenate_batches(ja_train_encoding)
val_labels, _ = concatenate_batches(ja_test_encoding)

In [9]:
from torch.utils.data import DataLoader, TensorDataset

# create TensorDataset
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)

# create DataLoader
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# check DataLoader
for batch in train_dataloader:
    input_ids, attention_masks, labels = batch
    print("Input IDs:", input_ids)
    print("Attention Masks:", attention_masks)
    print("Labels:", labels)
    break  # 最初のバッチのみ表示

Input IDs: tensor([[  101, 26742, 10151,  ...,     0,     0,     0],
        [  101,   146, 73871,  ...,     0,     0,     0],
        [  101, 33115,   117,  ...,     0,     0,     0],
        ...,
        [  101, 10117, 28502,  ...,     0,     0,     0],
        [  101, 43958, 65188,  ...,     0,     0,     0],
        [  101, 14535, 10192,  ...,     0,     0,     0]], device='cuda:0')
Attention Masks: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')
Labels: tensor([[  101,  1993, 13894,  ...,     0,     0,     0],
        [  101,  5949,  1947,  ...,     0,     0,     0],
        [  101, 49991, 12236,  ...,     0,     0,     0],
        ...,
        [  101,  1932, 27058,  ...,     0,     0,     0],
        [  101,  1911, 77610,  ...,     0,     0,     0],
        [  101,  7735,  7421,  ...,     0,     

In [10]:
from torch.cuda.amp import autocast, GradScaler

num_epochs = 3

optimizer = torch.optim.AdamW(bert_masked_model.parameters(), lr=5e-5)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
scaler = torch.cuda.amp.GradScaler()

In [11]:
from tqdm import tqdm

for epoch in range(num_epochs):
    bert_masked_model.train()
    total_loss = 0

    loop= tqdm(train_dataloader, leave=True)
    for batch in loop:
        input_ids, attention_masks, labels = batch
        input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)

        optimizer.zero_grad()

        # apply AMP (Mixed-Precision Training)
        with torch.cuda.amp.autocast():
          outputs = bert_masked_model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)
          loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        #loss.backward()
        #optimizer.step()
        #scheduler.step()

        total_loss += loss.item()

        # represent progress and average loss of each epoch
        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=total_loss/len(train_dataloader))

        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Train Loss: {avg_train_loss}")

    bert_masked_model.eval()
    total_eval_loss = 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_masks, labels = batch
            input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)

            outputs = bert_masked_model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)
            loss = outputs.loss
            total_eval_loss += loss.item()

    avg_val_loss = total_eval_loss / len(val_dataloader)
    print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss}")

Epoch 1:   0%|          | 0/563 [00:01<?, ?it/s, loss=0.041]/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch 1: 100%|██████████| 563/563 [02:36<00:00,  3.60it/s, loss=2.85]


Epoch 1, Train Loss: 2.853520224191791
Epoch 1, Validation Loss: 2.29830911613646


Epoch 2: 100%|██████████| 563/563 [02:37<00:00,  3.58it/s, loss=2.22]


Epoch 2, Train Loss: 2.217212956921457
Epoch 2, Validation Loss: 2.210039350721571


Epoch 3: 100%|██████████| 563/563 [02:37<00:00,  3.59it/s, loss=2.11]


Epoch 3, Train Loss: 2.108117797658456
Epoch 3, Validation Loss: 2.18773217049856


In [12]:
model_save_path = '/content/drive/MyDrive/machine_learning/bert_masked_model_sampleData.pth'
torch.save(bert_masked_model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to /content/drive/MyDrive/machine_learning/bert_masked_model_sampleData.pth


In [13]:
model_load_path = '/content/drive/MyDrive/machine_learning/bert_masked_model_sampleData.pth'
loaded_model = BertForMaskedLM.from_pretrained(model_name, config=config).to(device)
loaded_model.load_state_dict(torch.load(model_load_path, map_location=device))
print(f"Model loaded from {model_load_path}")


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded from /content/drive/MyDrive/machine_learning/bert_masked_model_sampleData.pth


In [14]:
sample_texts = ["This is a sample sentence for translation."]
encoded_input = tokenizer(sample_texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt").to(device)

loaded_model.eval()
with torch.no_grad():
    outputs = loaded_model(**encoded_input)
    predictions = outputs.logits.argmax(dim=-1)

decoded_output = tokenizer.batch_decode(predictions, skip_special_tokens=True)
for i, text in enumerate(sample_texts):
    print(f"Original: {text}")
    print(f"Translated: {decoded_output[i]}")

Original: This is a sample sentence for translation.
Translated: これは は の の 語 の 訳 の
